In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import os
import tensorflow as tf

In [2]:
import pathlib
data_folder = pathlib.Path('../input/flowers-recognition/flowers')
flower_folders = []
Labels = []
for item in data_folder.iterdir():
    flower_folders.append(item)
    # フォルダ名からラベルを取得
    Labels.append(str(item).split('/')[-1])

label_dictionary = dict(zip(Labels, np.arange(0,len(Labels))))
print(label_dictionary)

# .jpgのファイル数をカウントして、ndarrayを初期化したい
total_images_number = 0
for folder_path in flower_folders:
    for file_path in folder_path.iterdir():
        if os.path.splitext(file_path)[-1] == '.jpg':
            total_images_number += 1
print(total_images_number)

{'dandelion': 0, 'daisy': 1, 'flowers': 2, 'sunflower': 3, 'tulip': 4, 'rose': 5}
4323


In [3]:
image_size=256
X = np.empty([total_images_number,image_size, image_size, 3])
y = np.empty([0])
count = 0
from PIL import Image
for folder_path in flower_folders:
    for file_path in folder_path.iterdir():
        if os.path.splitext(file_path)[-1] == '.jpg':
            image = Image.open(file_path)
            # 画像データをリサイズし、ndarrayへ変換
            image_array = np.asarray(image.resize((image_size, image_size), Image.NEAREST))
            image.close()
            X[count,:,:,:] = image_array
            # 末尾に(label列に)ディクショナリからの値を追加
            folder_name = os.path.basename(os.path.dirname(file_path))
            y = np.append(y,label_dictionary[folder_name])
            count += 1
    print(str(folder_name) + ' Completed:')

dandelion Completed:
daisy Completed:
daisy Completed:
sunflower Completed:
tulip Completed:
rose Completed:


In [4]:
# ndarrayをtensorへ変換して、テストデータとする
X_tensor = tf.convert_to_tensor(X)

In [6]:
# CNNの初期化。単純なSequentialモデルで作成。
cnn = tf.keras.models.Sequential()

# 畳み込み https://keras.io/ja/layers/convolutional/
# 128x128 RGB画像ではinput_shape=(128, 128, 3)となります．
# 最初のレイヤーだけは、入り口となる入力シェイプが必要
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding="same", activation="relu", input_shape=[image_size, image_size, 3], data_format='channels_last'))

# Poolingにより、ダウンサンプリング
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2, padding='valid'))

# 2層目の中間層を追加
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding="same", activation="relu"))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2, padding='valid'))

# Flattening
cnn.add(tf.keras.layers.Flatten())

# 接続
# unitsは   レイヤーの出力形状
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

# 出力層
#cnn.add(tf.keras.layers.Dense(units=10, activation='sigmoid'))
cnn.add(tf.keras.layers.Dense(6, activation='softmax'))

# Compiling the CNN
# 評価関数の選定 -> https://keras.io/ja/metrics/
cnn.compile(optimizer = 'adam', loss = tf.keras.losses.sparse_categorical_crossentropy, metrics = ['accuracy'])

answer = cnn.fit(X_tensor, y, epochs = 10, batch_size = 20)

Epoch 1/10
217/217 [==============================] - 200s 924ms/step - loss: 73.0330 - accuracy: 0.2750
Epoch 2/10
217/217 [==============================] - 200s 923ms/step - loss: 1.3147 - accuracy: 0.4874
Epoch 3/10
217/217 [==============================] - 205s 944ms/step - loss: 0.8583 - accuracy: 0.6768
Epoch 4/10
217/217 [==============================] - 202s 930ms/step - loss: 0.5333 - accuracy: 0.8154
Epoch 5/10
217/217 [==============================] - 201s 927ms/step - loss: 0.5039 - accuracy: 0.8256
Epoch 6/10
217/217 [==============================] - 200s 923ms/step - loss: 0.4559 - accuracy: 0.8612
Epoch 7/10
217/217 [==============================] - 200s 920ms/step - loss: 0.3780 - accuracy: 0.8862
Epoch 8/10
217/217 [==============================] - 199s 915ms/step - loss: 0.3510 - accuracy: 0.8920
Epoch 9/10
217/217 [==============================] - 200s 922ms/step - loss: 0.2834 - accuracy: 0.9144
Epoch 10/10
217/217 [==============================] - 204s 938

In [ ]:
cnn.evaluate(X_tensor, y)